In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/train.csv")
val=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/test.csv")
sub=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv")

In [3]:
#zero, one =np.bincount(df['Class'])

In [4]:
#zero_df=df.loc[df['Class']==0]
#one_df=df.loc[df['Class']==1]

In [5]:
#zero_df=zero_df.dropna()
#zero_df.shape

In [6]:
#one_df=one_df.fillna(df.median())
#zero_df=zero_df.fillna(df.median())

In [7]:
#zero_df=zero_df.drop(zero_df.sample(106).index)#

In [8]:
#zero_df.shape

In [9]:
#data=pd.concat([one_df,zero_df])

In [10]:
#data=data.sample(frac=1,random_state=42)
#data = data.reset_index(drop=True)
data=df
data=data.fillna(data.median())

/tmp/ipykernel_20/4115765143.py:4: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data=data.fillna(data.median())


In [11]:
from sklearn.preprocessing import LabelEncoder

data['EJ'] = LabelEncoder().fit_transform(data['EJ'])
data=data.drop(['Id'], axis=1)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [12]:
val['EJ'] = LabelEncoder().fit_transform(val['EJ'])
val=val.drop(['Id'], axis=1)

In [13]:
X=data.drop(["Class"], axis=1)
y=data["Class"]

In [14]:
def competition_log_loss(y_true, y_pred):
    # y_true: correct labels 0, 1
    # y_pred: predicted probabilities of class=1
    #y_pred = y_pred.T[1]
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    w_0 = (N_0 + N_1) / N_0
    w_1 = (N_0 + N_1) / N_1
    # Calculate the predicted probabilities for each class
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    # Calculate the average log loss for each class
    sum_0 = -np.sum((1 - y_true) * np.log(p_0))
    sum_1 = -np.sum(y_true * np.log(p_1))
    to_output = ((w_0 / N_0) * sum_0 + (w_1 / N_1) * sum_1) / (w_0 + w_1)

    return to_output

In [15]:
from IPython.display import display
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold




def obj_xgb(trial, X,y):
    #x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.15)

    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000, step=50),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 1.0),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 0.001, 0.2),
        'objective': 'binary:logistic', 
        'eval_metric': 'logloss', 
    }

    custom_scorer = make_scorer(competition_log_loss, greater_is_better=False)
    model = xgb.XGBClassifier(**params)
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1121218)
    
    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        model.fit(X_train,y_train)
        preds = model.predict_proba(X_test)[:,1]
        #scores = cross_val_score(model, x, y, cv=kf,scoring=custom_scorer)
        cv_scores[idx] = competition_log_loss(y_test, preds)
 
        avg=cv_scores.mean()

    
    #y_pred = model.predict_proba(x_valid)[:, 1]
    

    #roc_auc = roc_auc_score(y_valid, y_pred)
    
    return avg


study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: obj_xgb(trial, X, y), n_trials=100)  # Pass X and y to the obj_xgb function


display(study)

print('Best trial:', study.best_trial.params)
print('Best brier:', study.best_trial.value)

[I 2023-07-29 09:49:01,708] A new study created in memory with name: no-name-fbf150a6-6b8a-4645-bb7f-bde151c8531f
/tmp/ipykernel_20/1395528414.py:21: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 1.0),
/tmp/ipykernel_20/1395528414.py:22: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
/tmp/ipykernel_20/1395528414.py:23: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 

Best trial: {'n_estimators': 250, 'max_depth': 8, 'learning_rate': 0.005369532425246009, 'subsample': 0.9874028482527533, 'colsample_bytree': 0.810137376733775, 'gamma': 0.0031465220096879437}
Best brier: 0.5765600463291045


In [16]:
print('Best trial:', study.best_trial.params)
print('Best balanced_log_loss:', study.best_trial.value)

Best trial: {'n_estimators': 250, 'max_depth': 8, 'learning_rate': 0.005369532425246009, 'subsample': 0.9874028482527533, 'colsample_bytree': 0.810137376733775, 'gamma': 0.0031465220096879437}
Best balanced_log_loss: 0.5765600463291045


In [17]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.15)

In [18]:
import xgboost as xgb
dtrain = xgb.DMatrix(x_train, label=y_train)
dtest = xgb.DMatrix(x_test, label=y_test)

In [19]:
params ={'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.11083666084808587, 'subsample': 0.6646998689138404, 'colsample_bytree': 0.9993276251354577, 'gamma': 0.09161209891211826}

In [20]:
model = xgb.train(params, dtrain,300)

[10:00:20] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.



In [21]:
y_pred = model.predict(dtest)

In [22]:
y_pred

array([ 8.59542098e-03,  6.81754854e-03, -4.35351245e-02,  6.88217521e-01,
        4.37828042e-02, -1.95481256e-02,  6.50753915e-01,  8.01873952e-03,
        3.06560454e-04,  8.07755347e-03, -2.37939674e-02, -1.10899741e-02,
        1.20681316e-01, -2.46171355e-02,  1.54031618e-02,  2.67311811e-01,
       -2.65595922e-03,  5.59714615e-01,  8.77616346e-01,  6.09713018e-01,
        3.42464656e-01, -4.64696204e-03,  2.13008430e-02,  3.48172560e-02,
       -1.00365141e-03,  8.84275198e-01,  7.88002431e-01, -2.26515718e-02,
        6.43200651e-02, -3.94191071e-02,  8.04249942e-01, -2.62568099e-03,
       -4.56307381e-02,  8.31700206e-01,  3.19912910e-01, -1.58393532e-02,
        5.88206828e-01, -1.12155369e-02,  1.03400916e-01, -3.21311131e-02,
       -8.98688193e-03,  8.53755057e-01,  7.22628608e-02,  8.53540182e-01,
        8.08391161e-03,  3.38536724e-02,  9.76336747e-02, -1.09946644e-02,
        9.50310886e-01, -2.37388778e-02, -1.09444195e-02, -2.14083567e-02,
        9.27583054e-02,  

In [23]:
from sklearn.metrics import accuracy_score

# Convert probabilities to binary predictions


y_pred_binary = [1 if p >= 0.5 else 0 for p in y_pred]

accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 94.62%


In [24]:
from sklearn.metrics import roc_auc_score

auc_score = roc_auc_score(y_test, y_pred)

In [25]:
auc_score

0.9295665634674922

In [26]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(y_test, y_pred_binary)

# Extract true negatives (TN), false positives (FP), false negatives (FN), and true positives (TP) from the confusion matrix
TN, FP, FN, TP = confusion.ravel()

# Calculate the false positive rate (FPR) and false negative rate (FNR)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)

# Print the false positive rate and false negative rate
print("False Positive Rate (FPR):", FPR)
print("False Negative Rate (FNR):", FNR)

False Positive Rate (FPR): 0.02631578947368421
False Negative Rate (FNR): 0.17647058823529413


In [27]:
competition_log_loss(y_test,y_pred)

2.0267005180811153

In [28]:
lb=np.zeros(val.shape[0])

In [29]:
dval = xgb.DMatrix(val, label=lb)


In [30]:
pred=model.predict(dval)

In [31]:
pred

array([0.19588824, 0.19588824, 0.19588824, 0.19588824, 0.19588824],
      dtype=float32)